# Import Packages

In [ ]:
library(opploansanalytics)
load.packages()

# Import Data

In [ ]:
getAdmethodList = function () {
    
    queryReporting(
    "
    select
        admethod
        , count(*) as volume
    from
        public.all_allapps
    where
        refi = 'N'
        and left(denygrp,1) > '2'
        and appldate >= '2019-10-01'::date
        and appldate < '2020-01-01'::date
    group by
        1
    order by
        2 desc
    "
    )
}

In [ ]:
getTestPayloads = function (admethod, start.date = '2019-12-01', end.date = '2020-01-01', limit = 100) {
    
    queryReporting(paste0(
    "
    select
        lde.lead_id
        , lde.lead_time at time zone 'America/Chicago' as lead_time
        , c_app.name
        , c_app.createddate at time zone 'America/Chicago' as appldate
        , c_am.name as admethod
        , lde.raw_lead
    from
        lde4.leads as lde
        inner join
            cloudlending.advertising_method as c_am
            on lde.partnerid = c_am.external_id
        left join
            cloudlending.applications as c_app
            on lde.lead_id = c_app.lde4_lead_id
    where
        c_am.name = '", admethod, "'
        and lde.lead_time at time zone 'America/Chicago' >= '", start.date, "'::date
        and lde.lead_time at time zone 'America/Chicago' < '", end.date, "'::date
    order by
        random()
    limit ", limit
    ))
    
}

# Process Data

In [ ]:
validateStructure = function () {}

In [ ]:
setGlobalVars = function () {

    setEvaluationVariables = function () {



        ##  Does the proposed payload contain any objects NOT expected?  ##
        expected.keys <<- c(
            'isProduction',
            'leadOfferId',
            'language',
            'socialSecurityNumber',
            'email',
            'stateCode',
            'grossMonthlyIncome',
            'currency',
            'firstName',
            'lastName',
            'dateOfBirth',
            'streetAddress',
            'city',
            'zip',
            'countryCode',
            'mobilePhone',
            'bankName',
            'abaRoutingNumber',
            'accountNumber',
            'accountType',
            'accountLength',
            'incomeType',
            'payrollType',
            'payrollFrequency',
            'lastPayrollDate',
            'nextPayrollDate',
            'employerName',
            'hireDate',
            'requestedLoanAmount'
    #         'personalInfo',
    #         'address',
    #         'homePhone',
    #         'bankInfo',
    #         'incomeInfo',
    #         'employmentInfo',
        )


        ##  What are the key value pairs?  ##
        values <<- '\\{?\\\".*?\\":\\s(?!=\\{)([^\\{]*?)[,\\}]'
        keys <<- '\\\"([^,]*?)\\\":'


        ##  What data types do we expect?  ##

        #   Always Bad   #
        quoted.null <<- '^\\\"null\\\"$'
        quoted.empty <<- '^\\\"(?:\\s+)?\\\"$'
        quoted.boolean <<- '^\\\"(?:true|false)\\\"$'

        quoted.string <<- '^\\\".*?\\\"$'
        quoted.numeric <<- '^\\\"\\d+\\\"$'
        quoted.zip <<- '^\\\"\\d{5}\\\"$'
        quoted.date <<- '^\\\"\\d{8}\\\"$'
        quoted.aba <<- '^\\\"\\d{9}\\\"$'
        quoted.ssn <<- '^\\\"(?:\\d{3})-?(?:\\d{2})-?(?:\\d{4})\\\"$'
        quoted.ssn.aba <<- '^\\\"\\d{9}\\\"$'
        quoted.phone <<- '^\\\"\\+?\\d{10,11}\\\"$'
        quoted.name <<- '^\\\"[a-zA-Z\\s]+\\\"$'
    #     quoted.email <<- '^\\\".+@.+\\.\\w+\\\"$'
        quoted.email <<- '^\\\"(?:[a-z0-9!#$%&\\\'*+/=?^_`{|}~-]+(?:\\.[a-z0-9!#$%&\\\'*+/=?^_`{|}~-]+)*|\\\"(?:[\\x01-\\x08\\x0b\\x0c\\x0e-\\x1f\\x21\\x23-\\x5b\\x5d-\\x7f]|\\\\[\\x01-\\x09\\x0b\\x0c\\x0e-\\x7f])*\\\")@(?:(?:[a-z0-9](?:[a-z0-9-]*[a-z0-9])?\\.)+[a-z0-9](?:[a-z0-9-]*[a-z0-9])?|\\[(?:(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\\.){3}(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?|[a-z0-9-]*[a-z0-9]:(?:[\\x01-\\x08\\x0b\\x0c\\x0e-\\x1f\\x21-\\x5a\\x53-\\x7f]|\\\\[\\x01-\\x09\\x0b\\x0c\\x0e-\\x7f])+)\\])\\\"$'
        quoted.state <<- '^\\\"[A-Z]{2}\\\"$'
    #     quoted.state <<- '^\\\"(?:AL|AK|AZ|AR|CA|CO|CT|DE|FL|GA|HI|ID|IL|IN|IA|KS|KY|LA|ME|MD|MA|MI|MN|MS|MO|MT|NE|NV|NH|NJ|NM|NY|NC|ND|OH|OK|OR|PA|RI|SC|SD|TN|TX|UT|VT|VA|WA|WV|WI|WY|DC)\\\"$'
        quoted.currency <<- '^\\\"USD\\\"$'
        quoted.blank <<- '^\\\"[\\s]*\\\"$'
        quoted.countryCode <<- '^\\\"\\s*(?:US|USA)\\s*\\\"$'
        quoted.accountNumber <<- '^\\\"\\d{6,17}\\\"$'

        unquoted.numeric <<- '[\\d\\.]+(?![\\\"@\\w\\s\\-])'
        unquoted.float.positive <<- '([^\\-\\s\\\"]\\d+\\.?\\d*[^\\\"\\,])'
        unquoted.decimal <<- '(?:\\d+\\.\\d+)+(?![\\\"@\\w\\s\\-])'
        unquoted.integer <<- '(?<=(?:\\:\\s))\\d+(?![\\\"@\\w\\s\\-\\.])'
        unquoted.boolean <<- '(?<!\\\")(?:true|false)(?!=\\\")'
        unquoted.null <<- '(?<!\\\")null(?!=\\\")'

        optional.quoted.float.positive <<- '(\\\"?[^\\-\\s]\\d+\\.?\\d*\\\"?)'

        expect.quoted.string <<- c(
            'campaignId',
            'leadOfferId',
            'streetAddress',
            'bankName'
        )

        expect.quoted.employerName <<- c(
            'employerName'
        )

        expect.quoted.numeric <<- c(
            'accountNumber'
        )

        expect.quoted.name <<- c(
            'firstName',
            'lastName',
            'city',
            'language',
            'countryCode',
            'incomeType',
            'payrollType'
        )

        expect.quoted.currency <<- c(
            'currency'
        )

        expect.quoted.zip <<- c(
            'zip'
        )

        expect.quoted.date.past <<- c(
            'dateOfBirth',
            'lastPayrollDate',
            'hireDate'
        )

        expect.quoted.date.future <<- c(
            'nextPayrollDate'
        )

        expect.quoted.ssn.aba <<- c(
            'socialSecurityNumber',
            'abaRoutingNumber'
        )

        expect.quoted.phone <<- c(
            'mobilePhone',
            'homePhone'
        )

        expect.quoted.email <<- c(
            'email'
        )

        expect.quoted.state <<- c(
            'stateCode'
        )

        expect.unquoted.boolean <<- c(
            'isProduction'
        )

        expect.unquoted.float.positive <<- c(
            'grossMonthlyIncome'
        )

        expect.unquoted.integer <<- c(
            'accountType',
            'accountLength',
            'payrollFrequency'
        )

        expect.optional.quoted.float.positive <<- c(
            'requestedLoanAmount'
        )

}
    setPartnerMappingLDE = function () {
        queryReporting(
        "
        select
            external_id
            , name
            , control_file
            , case 	when control_file = 'Leads02' then 'Non-Affiliate'
                    when control_file = 'Opp4' then 'Affiliate'
                    when control_file = 'Opp23' then 'Lite'
                    else 'Else'
                    end as lde_type
        from
            cloudlending.advertising_method
        "
        )
}

    setEvaluationVariables()
    partner.map <<- setPartnerMappingLDE()
    
}

In [ ]:
validateValues = function (key, value, admethod, ...) {
    
    args = list(...)
    validator = function (admethod) {
        
        if ( partner.map %>% filter(name == admethod) %>% .$lde_type %in% c('Affiliate') ) {
            
            ###  AFFILIATE  ###
            ldevalidate_isProduction <<- function (value) {

                response.vector = c()

                format.correct = expr(value %>% str_detect(unquoted.boolean))

                if (! eval(format.correct))
                    response.vector %<>% c(900)
                if (  eval(format.correct))
                    response.vector %<>% c(200)


                return (response.vector)

}                ## DONE - LDE Affl
            ldevalidate_leadOfferId <<- function (value) {

                response.vector = c()

                format.correct = expr(value %>% str_detect(quoted.string))

                if (! eval(format.correct))
                    response.vector %<>% c(901)
                if (  eval(format.correct))
                    response.vector %<>% c(200)


                return (response.vector)

}                 ## DONE - LDE Affl
            ldevalidate_clickId <<- function (value) {

                response.vector = c()

                format.correct = expr(value %>% str_detect(quoted.string))

                if (! eval(format.correct))
                    response.vector %<>% c(901)
                if (  eval(format.correct))
                    response.vector %<>% c(200)


                return (response.vector)

}                     ## DONE - LDE Affl
            ldevalidate_language <<- function (value) {

                response.vector = c()

                format.correct = expr(value %>% str_detect(quoted.name))

                if (! eval(format.correct))
                    response.vector %<>% c(902)
                if (  eval(format.correct))
                    response.vector %<>% c(200)


                return (response.vector)

}                    ## DONE - LDE Affl
            ldevalidate_socialSecurityNumber <<- function (value) {

                response.vector = c()

                format.correct = expr(value %>% str_detect(quoted.ssn))
                valid.first3 = expr(
                    ! value %>%
                        str_detect(
                            pattern = regex(
                                '^\\\"(\\d{3})-?(?:\\d{2})-?(?:\\d{4})\\\"$'
                            )
                        ) %in% c('666', '000')
                )
                valid.second3 = expr(
                    ! value %>%
                        str_detect(
                            pattern = regex(
                                '^\\\"(?:\\d{3})-?(\\d{2})-?(?:\\d{4})\\\"$'
                            )
                        ) %in% c('0')
                )
                valid.last4 = expr(
                    ! value %>%
                        str_detect(
                            pattern = regex(
                                '^\\\"(?:\\d{3})-?(?:\\d{2})-?(\\d{4})\\\"$'
                            )
                        ) %in% c('0000')
                )




                if (! eval(format.correct) |
                    ! eval(valid.first3) |
                    ! eval(valid.second3) |
                    ! eval(valid.last4) )
                    response.vector %<>% c(315)

                if (  eval(format.correct) &
                      eval(valid.first3) &
                      eval(valid.second3) &
                      eval(valid.last4) )
                    response.vector %<>% c(200)


                return (response.vector)

}        ## DONE - LDE Affl
            ldevalidate_email <<- function (value) {

                response.vector = c()

                format.correct = expr(value %>% str_detect(quoted.email))
                email.not.mil = expr(
                    ! value %>% 
                        str_detect(
                            pattern = regex(
                                '^\\\".*@.*\\.mil\\\"$'
                            )
                        )
                )

                if (! eval(format.correct))
                    response.vector %<>% c(313)
                if (! eval(email.not.mil))
                    response.vector %<>% c(313)
                if (  eval(format.correct) &
                      eval(email.not.mil) )
                    response.vector %<>% c(200)


                return (response.vector)

}                       ## DONE - LDE Affl
            ldevalidate_stateCode <<- function (value) {

                response.vector = c()

                format.correct = expr(value %>% str_detect(quoted.state))
                valid.state = expr(
                    value %>%
                        str_detect(
                            pattern = regex(
                                '^\\\"(?:AL|AK|AZ|AR|CA|CO|CT|DE|FL|GA|HI|ID|IL|IN|IA|KS|KY|LA|ME|MD|MA|MI|MN|MS|MO|MT|NE|NV|NH|NJ|NM|NY|NC|ND|OH|OK|OR|PA|RI|SC|SD|TN|TX|UT|VT|VA|WA|WV|WI|WY|DC)\\\"$'
                            )
                        )
                )

                if (! eval(format.correct))
                    response.vector %<>% c(905)
                if (! eval(valid.state))
                    response.vector %<>% c(402)
                if (  eval(format.correct) &
                      eval(valid.state))
                    response.vector %<>% c(200)


                return (response.vector)

}                   ## DONE - LDE Affl
            ldevalidate_grossMonthlyIncome <<- function (value) {

                response.vector = c()

                format.correct = expr(value %>% str_detect(unquoted.float.positive))

                if (! eval(format.correct))
                    response.vector %<>% c(333)
                if (  eval(format.correct))
                    response.vector %<>% c(200)


                return (response.vector)

}          ## DONE - LDE Affl
            ldevalidate_currencyCode <<- function (value) {

                response.vector = c()

                format.correct = expr(value %>% str_detect(quoted.currency))

                if (! eval(format.correct))
                    response.vector %<>% c(907)
                if (  eval(format.correct))
                    response.vector %<>% c(200)


                return (response.vector)

}                ## DONE - LDE Affl
            ldevalidate_currency <<- function (value) {

                response.vector = c()

                format.correct = expr(value %>% str_detect(quoted.currency))

                if (! eval(format.correct))
                    response.vector %<>% c(907)
                if (  eval(format.correct))
                    response.vector %<>% c(200)


                return (response.vector)

}                    ## DONE - LDE Affl
            ldevalidate_firstName <<- function (value) {

                response.vector = c()

                format.correct = expr(value %>% str_detect(quoted.name))

                if (! eval(format.correct))
                    response.vector %<>% c(908)
                if (  eval(format.correct))
                    response.vector %<>% c(200)


                return (response.vector)

}                   ## DONE - LDE Affl
            ldevalidate_lastName <<- function (value) {

                response.vector = c()

                format.correct = expr(value %>% str_detect(quoted.name))

                if (! eval(format.correct))
                    response.vector %<>% c(909)
                if (  eval(format.correct))
                    response.vector %<>% c(200)


                return (response.vector)

}                    ## DONE - LDE Affl
            ldevalidate_dateOfBirth <<- function (value) {

                response.vector = c()

                format.correct = expr(value %>% str_detect(quoted.date))
                length.correct = expr(value %>% str_remove_all("\"") %>% as.integer() %>% nchar() == 8)
                date.in.past = expr(
                    value %>% str_remove_all("\"") %>% as.integer() <= args$present.time
                )

                if (! eval(format.correct) |
                    ! eval(length.correct) )
                    response.vector %<>% c(319)

                if (  eval(format.correct) &
                      eval(length.correct) &
                    ! eval(date.in.past) )
                    response.vector %<>% c(624)

                if (  eval(format.correct) &
                      eval(length.correct) &
                      eval(date.in.past) )
                    response.vector %<>% c(200)



                return (response.vector)

}                 ## DONE - LDE Affl
            ldevalidate_streetAddress <<- function (value) {

                response.vector = c()

                format.correct = expr(value %>% str_detect(quoted.string))
                not.pobox = expr(
                    !value %>% str_detect(
                        pattern = regex(
                            "(?i)^\\\".*p[.]?o[.]?[\\s]*box.*\\\"$"
                        )
                    )
                )
                not.blank = expr(
                    !(
                        value %>% str_detect(unquoted.null) |
                        value %>% str_detect(quoted.blank) |
                        value %>% str_detect(unquoted.boolean)
                    )
                )

                if (! eval(format.correct))
                    response.vector %<>% c(911)
                if (! eval(not.blank))
                    response.vector %<>% c(342)
                if (! eval(not.pobox))
                    response.vector %<>% c(608)
                if (  eval(format.correct))
                    response.vector %<>% c(200)


                return (response.vector)

}               ## DONE - LDE Affl
            ldevalidate_city <<- function (value) {

                response.vector = c()

                format.correct = expr(value %>% str_detect(quoted.name))
                not.blank = expr(
                    !(
                        value %>% str_detect(unquoted.null) |
                        value %>% str_detect(quoted.blank) |
                        value %>% str_detect(unquoted.boolean)
                    )
                )

                if (! eval(format.correct))
                    response.vector %<>% c(912)
                if (! eval(not.blank))
                    response.vector %<>% c(342)
                if (  eval(format.correct))
                    response.vector %<>% c(200)


                return (response.vector)

}                        ## DONE - LDE Affl
            ldevalidate_zip <<- function (value) {

                response.vector = c()

                format.correct = expr(value %>% str_detect(quoted.zip))
                not.blank = expr(
                    !(
                        value %>% str_detect(unquoted.null) |
                        value %>% str_detect(quoted.blank) |
                        value %>% str_detect(unquoted.boolean)
                    )
                )

                if (! eval(format.correct))
                    response.vector %<>% c(913)
                if (! eval(not.blank))
                    response.vector %<>% c(342)
                if (  eval(format.correct))
                    response.vector %<>% c(200)


                return (response.vector)

}                         ## DONE - LDE Affl
            ldevalidate_countryCode <<- function (value) {

                response.vector = c()

                format.correct = expr(value %>% str_detect(quoted.countryCode))

                if (! eval(format.correct))
                    response.vector %<>% c(403)
                if (  eval(format.correct))
                    response.vector %<>% c(200)


                return (response.vector)

}                 ## DONE - LDE Affl
            ldevalidate_mobilePhone <<- function (value) {

                response.vector = c()

                format.correct = expr(value %>% str_detect(quoted.phone))

                if (! eval(format.correct))
                    response.vector %<>% c(915)
                if (  eval(format.correct))
                    response.vector %<>% c(200)


                return (response.vector)

}                 ## DONE - LDE Affl
            ldevalidate_homePhone <<- function (value) {

                response.vector = c()

                format.correct = expr(value %>% str_detect(quoted.phone))

                if (! eval(format.correct))
                    response.vector %<>% c(915)
                if (  eval(format.correct))
                    response.vector %<>% c(200)


                return (response.vector)

}                   ## DONE - LDE Affl
            ldevalidate_bankName <<- function (value) {

                response.vector = c()

                format.correct = expr(value %>% str_detect(quoted.string))

                if (! eval(format.correct))
                    response.vector %<>% c(916)
                if (  eval(format.correct))
                    response.vector %<>% c(200)


                return (response.vector)

}                    ## DONE - LDE Affl
            ldevalidate_abaRoutingNumber <<- function (value) {

                response.vector = c()

                format.correct = expr(value %>% str_detect(quoted.aba))
                checksum.pass = expr(
                    (
                        value %>% substr(1,1) %>% as.integer() * 3 +
                        value %>% substr(2,2) %>% as.integer() * 7 +
                        value %>% substr(3,3) %>% as.integer() * 1 +
                        value %>% substr(4,4) %>% as.integer() * 3 +
                        value %>% substr(5,5) %>% as.integer() * 7 +
                        value %>% substr(6,6) %>% as.integer() * 1 +
                        value %>% substr(7,7) %>% as.integer() * 3 +
                        value %>% substr(8,8) %>% as.integer() * 7 +
                        value %>% substr(9,9) %>% as.integer() * 1
                    ) %% 10 == 0
                )

                if (! eval(format.correct))
                    response.vector %<>% c(711)
                if (  eval(format.correct) &
                    ! eval(checksum.pass) )
                    response.vector %<>% c(999)
                if (  eval(format.correct) &
                      eval(checksum.pass) )
                    response.vector %<>% c(200)


                return (response.vector)

}            ## DONE - LDE Affl
            ldevalidate_accountNumber <<- function (value) {

                response.vector = c()

                format.correct = expr(value %>% str_detect(quoted.accountNumber))

                if (! eval(format.correct))
                    response.vector %<>% c(710)
                if (  eval(format.correct) )
                    response.vector %<>% c(200)


                return (response.vector)

}               ## DONE - LDE Affl
            ldevalidate_accountType <<- function (value) {

                response.vector = c()

                format.correct = expr(value %>% str_detect(unquoted.integer))

                if (! eval(format.correct))
                    response.vector %<>% c(712)
                if (  eval(format.correct))
                    response.vector %<>% c(200)


                return (response.vector)

}                 ## DONE - LDE Affl
            ldevalidate_accountLength <<- function (value) {

                response.vector = c()

                format.correct = expr(value %>% str_detect(unquoted.integer))

                if (! eval(format.correct))
                    response.vector %<>% c(713)
                if (  eval(format.correct))
                    response.vector %<>% c(200)


                return (response.vector)

}               ## DONE - LDE Affl
            ldevalidate_incomeType <<- function (value) {

                response.vector = c()

                format.correct = expr(value %>% str_detect(quoted.name))

                if (! eval(format.correct))
                    response.vector %<>% c(921)
                if (  eval(format.correct))
                    response.vector %<>% c(200)


                return (response.vector)

}                  ## DONE - LDE Affl
            ldevalidate_payrollType <<- function (value) {

                response.vector = c()

                format.correct = expr(value %>% str_detect(quoted.name))

                if (! eval(format.correct))
                    response.vector %<>% c(922)
                if (  eval(format.correct))
                    response.vector %<>% c(200)


                return (response.vector)

}                 ## DONE - LDE Affl
            ldevalidate_payrollFrequency <<- function (value) {

                response.vector = c()

                format.correct = expr(value %>% str_detect(unquoted.integer))

                if (! eval(format.correct))
                    response.vector %<>% c(706)
                if (  eval(format.correct))
                    response.vector %<>% c(200)


                return (response.vector)

}            ## DONE - LDE Affl
            ldevalidate_lastPayrollDate <<- function (value) {

                # can be unquoted null OR (date format AND in the past)
                response.vector = c()

                format.isnull = expr(value %>% str_detect(unquoted.null))
                format.correct = expr(value %>% str_detect(quoted.date))
                length.correct = expr(value %>% str_remove_all("\"") %>% as.integer() %>% nchar() == 8)
                date.in.past = expr(
                    value %>% str_remove_all("\"") %>% as.integer() <= args$present.time
                )

                if (  ! eval(format.isnull) &
                     (! eval(format.correct) | ! eval(length.correct))  )
                    response.vector %<>% c(707)

                if ( ! eval(format.isnull) &
                      (eval(format.correct) & eval(length.correct)) & ! eval(date.in.past) )
                    response.vector %<>% c(707)

                if ( eval(format.isnull) |
                    (eval(format.correct) & eval(length.correct)) & eval(date.in.past) )
                    response.vector %<>% c(200)


                return (response.vector)

}             ## DONE - LDE Affl
            ldevalidate_nextPayrollDate <<- function (value) {

                # can be unquoted null OR (date format AND in the future)
                response.vector = c()

                format.isnull = expr(value %>% str_detect(unquoted.null))
                format.correct = expr(value %>% str_detect(quoted.date))
                length.correct = expr(value %>% str_remove_all("\"") %>% as.integer() %>% nchar() == 8)
                date.in.future = expr(
                    value %>% str_remove_all("\"") %>% as.integer() > args$present.time
                )

                if (  ! eval(format.isnull) &
                     (! eval(format.correct) | ! eval(length.correct))  )
                    response.vector %<>% c(708)

                if ( ! eval(format.isnull) &
                      (eval(format.correct) & eval(length.correct)) & ! eval(date.in.future) )
                    response.vector %<>% c(708)

                if ( eval(format.isnull) |
                    (eval(format.correct) & eval(length.correct)) & eval(date.in.future) )
                    response.vector %<>% c(200)


                return (response.vector)

}             ## DONE - LDE Affl
            ldevalidate_employerName <<- function (value) {

                response.vector = c()

                format.correct = expr(value %>% str_detect(quoted.string))
                different.from.bankName = expr(
                    value != args$bankName || length(args$bankName) == 0
                )

                if (! eval(format.correct))
                    response.vector %<>% c(926)
                if (! eval(different.from.bankName))
                    response.vector %<>% c(610)
                if (  eval(format.correct) &
                      eval(different.from.bankName) )
                    response.vector %<>% c(200)


                return (response.vector)

}                ## DONE - LDE Affl
            ldevalidate_hireDate <<- function (value) {

                # can be unquoted null OR (date format AND in the past)
                response.vector = c()

                format.isnull = expr(value %>% str_detect(unquoted.null))
                format.correct = expr(value %>% str_detect(quoted.date))
                length.correct = expr(value %>% str_remove_all("\"") %>% as.integer() %>% nchar() == 8)
                date.in.past = expr(
                    value %>% str_remove_all("\"") %>% as.integer() <= args$present.time
                )

                if (  ! eval(format.isnull) &
                     (! eval(format.correct) | ! eval(length.correct))  )
                    response.vector %<>% c(319)

                if ( ! eval(format.isnull) &
                      (eval(format.correct) & eval(length.correct)) & ! eval(date.in.past) )
                    response.vector %<>% c(319)

                if ( eval(format.isnull) |
                    (eval(format.correct) & eval(length.correct)) & eval(date.in.past) )
                    response.vector %<>% c(200)


                return (response.vector)

            }                    ## DONE - LDE Affl
            ldevalidate_requestedLoanAmount <<- function (value) {

                response.vector = c()

                format.correct = expr(value %>% str_detect(optional.quoted.float.positive))

                if (! eval(format.correct))
                    response.vector %<>% c(714)
                if (  eval(format.correct))
                    response.vector %<>% c(200)


                return (response.vector)

            }         ## DONE - LDE Affl
            ldevalidate_loanPurpose <<- function (value) {

                response.vector = c()

                format.correct = expr(value %>% str_detect(quoted.string))

                if (! eval(format.correct))
                    response.vector %<>% c(929)
                if (  eval(format.correct))
                    response.vector %<>% c(200)


                return (response.vector)

}                 ## DONE - LDE Affl
            
            
        } else {
            
            ###  LDE LITE, LDE NON-AFFILIATE  ###
            ldevalidate_isProduction <<- function (value) {

                response.vector = c()

                format.correct = expr(value %>% str_detect(unquoted.boolean))

                if (! eval(format.correct))
                    response.vector %<>% c(900)
                if (  eval(format.correct))
                    response.vector %<>% c(200)


                return (response.vector)

}                ## DONE - LDE Lite
            ldevalidate_leadOfferId <<- function (value) {

                response.vector = c()

                format.correct = expr(value %>% str_detect(quoted.string))

                if (! eval(format.correct))
                    response.vector %<>% c(901)
                if (  eval(format.correct))
                    response.vector %<>% c(200)


                return (response.vector)

}                 ## DONE - LDE Lite
            ldevalidate_clickId <<- function (value) {

                response.vector = c()

                format.correct = expr(value %>% str_detect(quoted.string))

                if (! eval(format.correct))
                    response.vector %<>% c(901)
                if (  eval(format.correct))
                    response.vector %<>% c(200)


                return (response.vector)

}                     ## DONE - LDE Lite
            ldevalidate_language <<- function (value) {

                response.vector = c()

                format.correct = expr(value %>% str_detect(quoted.name))

                if (! eval(format.correct))
                    response.vector %<>% c(902)
                if (  eval(format.correct))
                    response.vector %<>% c(200)


                return (response.vector)

}                    ## DONE - LDE Lite
            ldevalidate_socialSecurityNumber <<- function (value) {

                response.vector = c()

                format.correct = expr(value %>% str_detect(quoted.ssn))
                valid.first3 = expr(
                    ! value %>%
                        str_detect(
                            pattern = regex(
                                '^\\\"(\\d{3})-?(?:\\d{2})-?(?:\\d{4})\\\"$'
                            )
                        ) %in% c('666', '000')
                )
                valid.second3 = expr(
                    ! value %>%
                        str_detect(
                            pattern = regex(
                                '^\\\"(?:\\d{3})-?(\\d{2})-?(?:\\d{4})\\\"$'
                            )
                        ) %in% c('0')
                )
                valid.last4 = expr(
                    ! value %>%
                        str_detect(
                            pattern = regex(
                                '^\\\"(?:\\d{3})-?(?:\\d{2})-?(\\d{4})\\\"$'
                            )
                        ) %in% c('0000')
                )




                if (! eval(format.correct) |
                    ! eval(valid.first3) |
                    ! eval(valid.second3) |
                    ! eval(valid.last4) )
                    response.vector %<>% c(315)

                if (  eval(format.correct) &
                      eval(valid.first3) &
                      eval(valid.second3) &
                      eval(valid.last4) )
                    response.vector %<>% c(200)


                return (response.vector)

}        ## DONE - LDE Lite
            ldevalidate_email <<- function (value) {

                response.vector = c()

                format.correct = expr(value %>% str_detect(quoted.email))
                email.not.mil = expr(
                    ! value %>% 
                        str_detect(
                            pattern = regex(
                                '^\\\".*@.*\\.mil\\\"$'
                            )
                        )
                )

                if (! eval(format.correct))
                    response.vector %<>% c(313)
                if (! eval(email.not.mil))
                    response.vector %<>% c(313)
                if (  eval(format.correct) &
                      eval(email.not.mil) )
                    response.vector %<>% c(200)


                return (response.vector)

}                       ## DONE - LDE Lite
            ldevalidate_stateCode <<- function (value) {

                response.vector = c()

                format.correct = expr(value %>% str_detect(quoted.state))
                valid.state = expr(
                    value %>%
                        str_detect(
                            pattern = regex(
                                '^\\\"(?:AL|AK|AZ|AR|CA|CO|CT|DE|FL|GA|HI|ID|IL|IN|IA|KS|KY|LA|ME|MD|MA|MI|MN|MS|MO|MT|NE|NV|NH|NJ|NM|NY|NC|ND|OH|OK|OR|PA|RI|SC|SD|TN|TX|UT|VT|VA|WA|WV|WI|WY|DC)\\\"$'
                            )
                        )
                )

                if (! eval(format.correct))
                    response.vector %<>% c(905)
                if (! eval(valid.state))
                    response.vector %<>% c(402)
                if (  eval(format.correct) &
                      eval(valid.state))
                    response.vector %<>% c(200)


                return (response.vector)

}                   ## DONE - LDE Lite
            ldevalidate_grossMonthlyIncome <<- function (value) {

                response.vector = c()

                format.correct = expr(value %>% str_detect(unquoted.float.positive))

                if (! eval(format.correct))
                    response.vector %<>% c(333)
                if (  eval(format.correct))
                    response.vector %<>% c(200)


                return (response.vector)

}          ## DONE - LDE Lite
            ldevalidate_currencyCode <<- function (value) {

                response.vector = c()

                format.correct = expr(value %>% str_detect(quoted.currency))

                if (! eval(format.correct))
                    response.vector %<>% c(907)
                if (  eval(format.correct))
                    response.vector %<>% c(200)


                return (response.vector)

}                ## DONE - LDE Lite
            ldevalidate_currency <<- function (value) {

                response.vector = c()

                format.correct = expr(value %>% str_detect(quoted.currency))

                if (! eval(format.correct))
                    response.vector %<>% c(907)
                if (  eval(format.correct))
                    response.vector %<>% c(200)


                return (response.vector)

}                    ## DONE - LDE Lite
            ldevalidate_firstName <<- function (value) {

                response.vector = c()

                format.correct = expr(value %>% str_detect(quoted.name))

                if (! eval(format.correct))
                    response.vector %<>% c(908)
                if (  eval(format.correct))
                    response.vector %<>% c(200)


                return (response.vector)

}                   ## DONE - LDE Lite
            ldevalidate_lastName <<- function (value) {

                response.vector = c()

                format.correct = expr(value %>% str_detect(quoted.name))

                if (! eval(format.correct))
                    response.vector %<>% c(909)
                if (  eval(format.correct))
                    response.vector %<>% c(200)


                return (response.vector)

}                    ## DONE - LDE Lite
            ldevalidate_dateOfBirth <<- function (value) {

                response.vector = c()

                format.correct = expr(value %>% str_detect(quoted.date))
                length.correct = expr(value %>% str_remove_all("\"") %>% as.integer() %>% nchar() == 8)
                date.in.past = expr(
                    value %>% str_remove_all("\"") %>% as.integer() <= args$present.time
                )

                if (! eval(format.correct) |
                    ! eval(length.correct) )
                    response.vector %<>% c(319)

                if (  eval(format.correct) &
                      eval(length.correct) &
                    ! eval(date.in.past) )
                    response.vector %<>% c(624)

                if (  eval(format.correct) &
                      eval(length.correct) &
                      eval(date.in.past) )
                    response.vector %<>% c(200)



                return (response.vector)

}                 ## DONE - LDE Lite
            ldevalidate_streetAddress <<- function (value) {

                response.vector = c()

                format.correct = expr(value %>% str_detect(quoted.string))
                not.pobox = expr(
                    !value %>% str_detect(
                        pattern = regex(
                            "(?i)^\\\".*p[.]?o[.]?[\\s]*box.*\\\"$"
                        )
                    )
                )
                not.blank = expr(
                    !(
                        value %>% str_detect(unquoted.null) |
                        value %>% str_detect(quoted.blank) |
                        value %>% str_detect(unquoted.boolean)
                    )
                )

                if (! eval(format.correct))
                    response.vector %<>% c(911)
                if (! eval(not.blank))
                    response.vector %<>% c(342)
                if (! eval(not.pobox))
                    response.vector %<>% c(608)
                if (  eval(format.correct))
                    response.vector %<>% c(200)


                return (response.vector)

}               ## DONE - LDE Lite
            ldevalidate_city <<- function (value) {

                response.vector = c()

                format.correct = expr(value %>% str_detect(quoted.name))
                not.blank = expr(
                    !(
                        value %>% str_detect(unquoted.null) |
                        value %>% str_detect(quoted.blank) |
                        value %>% str_detect(unquoted.boolean)
                    )
                )

                if (! eval(format.correct))
                    response.vector %<>% c(912)
                if (! eval(not.blank))
                    response.vector %<>% c(342)
                if (  eval(format.correct))
                    response.vector %<>% c(200)


                return (response.vector)

}                        ## DONE - LDE Lite
            ldevalidate_zip <<- function (value) {

                response.vector = c()

                format.correct = expr(value %>% str_detect(quoted.zip))
                not.blank = expr(
                    !(
                        value %>% str_detect(unquoted.null) |
                        value %>% str_detect(quoted.blank) |
                        value %>% str_detect(unquoted.boolean)
                    )
                )

                if (! eval(format.correct))
                    response.vector %<>% c(913)
                if (! eval(not.blank))
                    response.vector %<>% c(342)
                if (  eval(format.correct))
                    response.vector %<>% c(200)


                return (response.vector)

}                         ## DONE - LDE Lite
            ldevalidate_countryCode <<- function (value) {

                response.vector = c()

                format.correct = expr(value %>% str_detect(quoted.countryCode))

                if (! eval(format.correct))
                    response.vector %<>% c(403)
                if (  eval(format.correct))
                    response.vector %<>% c(200)


                return (response.vector)

}                 ## DONE - LDE Lite
            ldevalidate_mobilePhone <<- function (value) {

                response.vector = c()

                format.correct = expr(value %>% str_detect(quoted.phone))

                if (! eval(format.correct))
                    response.vector %<>% c(915)
                if (  eval(format.correct))
                    response.vector %<>% c(200)


                return (response.vector)

}                 ## DONE - LDE Lite
            ldevalidate_homePhone <<- function (value) {

                response.vector = c()

                format.correct = expr(value %>% str_detect(quoted.phone))

                if (! eval(format.correct))
                    response.vector %<>% c(915)
                if (  eval(format.correct))
                    response.vector %<>% c(200)


                return (response.vector)

}                   ## DONE - LDE Lite
            ldevalidate_bankName <<- function (value) {

                response.vector = c()

                format.correct = expr(value %>% str_detect(quoted.string))

                if (! eval(format.correct))
                    response.vector %<>% c(916)
                if (  eval(format.correct))
                    response.vector %<>% c(200)


                return (response.vector)

}                    ## DONE - LDE Lite
            ldevalidate_abaRoutingNumber <<- function (value) {

                response.vector = c()

                format.correct = expr(value %>% str_detect(quoted.aba))
                checksum.pass = expr(
                    (
                        value %>% substr(1,1) %>% as.integer() * 3 +
                        value %>% substr(2,2) %>% as.integer() * 7 +
                        value %>% substr(3,3) %>% as.integer() * 1 +
                        value %>% substr(4,4) %>% as.integer() * 3 +
                        value %>% substr(5,5) %>% as.integer() * 7 +
                        value %>% substr(6,6) %>% as.integer() * 1 +
                        value %>% substr(7,7) %>% as.integer() * 3 +
                        value %>% substr(8,8) %>% as.integer() * 7 +
                        value %>% substr(9,9) %>% as.integer() * 1
                    ) %% 10 == 0
                )

                if (! eval(format.correct))
                    response.vector %<>% c(917)
                if (  eval(format.correct) &
                    ! eval(checksum.pass) )
                    response.vector %<>% c(999)
                if (  eval(format.correct) &
                      eval(checksum.pass) )
                    response.vector %<>% c(200)


                return (response.vector)

}            ## DONE - LDE Lite
            ldevalidate_accountNumber <<- function (value) {

                response.vector = c()

                format.correct = expr(value %>% str_detect(quoted.accountNumber))
                is.null = expr(
                    value %>% str_detect(unquoted.null)
                )
                not.blank = expr(
                    !(
                        value %>% str_detect(quoted.blank) |
                        value %>% str_detect(unquoted.boolean)
                    )
                )

                if (! eval(is.null) & ! eval(format.correct))
                    response.vector %<>% c(605)
                if (! eval(is.null) & ! eval(not.blank) )
                    response.vector %<>% c(322)
                if (    eval(is.null) |
                     (! eval(is.null) & eval(format.correct) & eval(not.blank) ) )
                    response.vector %<>% c(200)


                return (response.vector)

}               ## DONE - LDE Lite
            ldevalidate_accountType <<- function (value) {

                response.vector = c()

                format.correct = expr(value %>% str_detect(unquoted.integer))

                if (! eval(format.correct))
                    response.vector %<>% c(919)
                if (  eval(format.correct))
                    response.vector %<>% c(200)


                return (response.vector)

}                 ## DONE - LDE Lite
            ldevalidate_accountLength <<- function (value) {

                response.vector = c()

                format.correct = expr(value %>% str_detect(unquoted.integer))

                if (! eval(format.correct))
                    response.vector %<>% c(920)
                if (  eval(format.correct))
                    response.vector %<>% c(200)


                return (response.vector)

}               ## DONE - LDE Lite
            ldevalidate_incomeType <<- function (value) {

                response.vector = c()

                format.correct = expr(value %>% str_detect(quoted.name))

                if (! eval(format.correct))
                    response.vector %<>% c(921)
                if (  eval(format.correct))
                    response.vector %<>% c(200)


                return (response.vector)

}                  ## DONE - LDE Lite
            ldevalidate_payrollType <<- function (value) {

                response.vector = c()

                format.correct = expr(value %>% str_detect(quoted.name))

                if (! eval(format.correct))
                    response.vector %<>% c(922)
                if (  eval(format.correct))
                    response.vector %<>% c(200)


                return (response.vector)

}                 ## DONE - LDE Lite
            ldevalidate_payrollFrequency <<- function (value) {

                response.vector = c()

                format.correct = expr(value %>% str_detect(unquoted.integer))

                if (! eval(format.correct))
                    response.vector %<>% c(923)
                if (  eval(format.correct))
                    response.vector %<>% c(200)


                return (response.vector)

}            ## DONE - LDE Lite
            ldevalidate_lastPayrollDate <<- function (value) {

                # can be unquoted null OR (date format AND in the past)
                response.vector = c()

                format.isnull = expr(value %>% str_detect(unquoted.null))
                format.correct = expr(value %>% str_detect(quoted.date))
                length.correct = expr(value %>% str_remove_all("\"") %>% as.integer() %>% nchar() == 8)
                date.in.past = expr(
                    value %>% str_remove_all("\"") %>% as.integer() <= args$present.time
                )

                if (  ! eval(format.isnull) &
                     (! eval(format.correct) | ! eval(length.correct))  )
                    response.vector %<>% c(319)

                if ( ! eval(format.isnull) &
                      (eval(format.correct) & eval(length.correct)) & ! eval(date.in.past) )
                    response.vector %<>% c(319)

                if ( eval(format.isnull) |
                    (eval(format.correct) & eval(length.correct)) & eval(date.in.past) )
                    response.vector %<>% c(200)


                return (response.vector)

}             ## DONE - LDE Lite
            ldevalidate_nextPayrollDate <<- function (value) {

                # can be unquoted null OR (date format AND in the future)
                response.vector = c()

                format.isnull = expr(value %>% str_detect(unquoted.null))
                format.correct = expr(value %>% str_detect(quoted.date))
                length.correct = expr(value %>% str_remove_all("\"") %>% as.integer() %>% nchar() == 8)
                date.in.future = expr(
                    value %>% str_remove_all("\"") %>% as.integer() > args$present.time
                )

                if (  ! eval(format.isnull) &
                     (! eval(format.correct) | ! eval(length.correct))  )
                    response.vector %<>% c(319)

                if ( ! eval(format.isnull) &
                      (eval(format.correct) & eval(length.correct)) & ! eval(date.in.future) )
                    response.vector %<>% c(319)

                if ( eval(format.isnull) |
                    (eval(format.correct) & eval(length.correct)) & eval(date.in.future) )
                    response.vector %<>% c(200)


                return (response.vector)

}             ## DONE - LDE Lite
            ldevalidate_employerName <<- function (value) {

                response.vector = c()

                format.correct = expr(value %>% str_detect(quoted.string))
                different.from.bankName = expr(
                    value != args$bankName || length(args$bankName) == 0
                )

                if (! eval(format.correct))
                    response.vector %<>% c(926)
                if (! eval(different.from.bankName))
                    response.vector %<>% c(610)
                if (  eval(format.correct) &
                      eval(different.from.bankName) )
                    response.vector %<>% c(200)


                return (response.vector)

}                ## DONE - LDE Lite
            ldevalidate_hireDate <<- function (value) {

                # can be unquoted null OR (date format AND in the past)
                response.vector = c()

                format.isnull = expr(value %>% str_detect(unquoted.null))
                format.correct = expr(value %>% str_detect(quoted.date))
                length.correct = expr(value %>% str_remove_all("\"") %>% as.integer() %>% nchar() == 8)
                date.in.past = expr(
                    value %>% str_remove_all("\"") %>% as.integer() <= args$present.time
                )

                if (  ! eval(format.isnull) &
                     (! eval(format.correct) | ! eval(length.correct))  )
                    response.vector %<>% c(319)

                if ( ! eval(format.isnull) &
                      (eval(format.correct) & eval(length.correct)) & ! eval(date.in.past) )
                    response.vector %<>% c(319)

                if ( eval(format.isnull) |
                    (eval(format.correct) & eval(length.correct)) & eval(date.in.past) )
                    response.vector %<>% c(200)


                return (response.vector)

            }                    ## DONE - LDE Lite
            ldevalidate_requestedLoanAmount <<- function (value) {

                response.vector = c()

                format.correct = expr(value %>% str_detect(optional.quoted.float.positive))

                if (! eval(format.correct))
                    response.vector %<>% c(928)
                if (  eval(format.correct))
                    response.vector %<>% c(200)


                return (response.vector)

            }         ## DONE - LDE Lite
            ldevalidate_loanPurpose <<- function (value) {

                response.vector = c()

                format.correct = expr(value %>% str_detect(quoted.string))

                if (! eval(format.correct))
                    response.vector %<>% c(929)
                if (  eval(format.correct))
                    response.vector %<>% c(200)


                return (response.vector)

}                 ## DONE - LDE Lite
            
        }
        
        ldevalidate_map_set = function () {
            list(
                isProduction = ldevalidate_isProduction,
                leadOfferId = ldevalidate_leadOfferId,
                click_id = ldevalidate_clickId,
                language = ldevalidate_language,
                socialSecurityNumber = ldevalidate_socialSecurityNumber,
                email = ldevalidate_email,
                stateCode = ldevalidate_stateCode,
                grossMonthlyIncome = ldevalidate_grossMonthlyIncome,
                currencyCode = ldevalidate_currencyCode,
                firstName = ldevalidate_firstName,
                lastName = ldevalidate_lastName,
                dateOfBirth = ldevalidate_dateOfBirth,
                streetAddress = ldevalidate_streetAddress,
                city = ldevalidate_city,
                zip = ldevalidate_zip,
                countryCode = ldevalidate_countryCode,
                mobilePhone = ldevalidate_mobilePhone,
                homePhone = ldevalidate_homePhone,
                bankName = ldevalidate_bankName,
                abaRoutingNumber = ldevalidate_abaRoutingNumber,
                accountNumber = ldevalidate_accountNumber,
                accountType = ldevalidate_accountType,
                accountLength = ldevalidate_accountLength,
                incomeType = ldevalidate_incomeType,
                payrollType = ldevalidate_payrollType,
                payrollFrequency = ldevalidate_payrollFrequency,
                lastPayrollDate = ldevalidate_lastPayrollDate,
                nextPayrollDate = ldevalidate_nextPayrollDate,
                employerName = ldevalidate_employerName,
                hireDate = ldevalidate_hireDate,
                requestedLoanAmount = ldevalidate_requestedLoanAmount,
                loanPurpose = ldevalidate_loanPurpose
            )
        }
        
        return (ldevalidate_map_set())
        
    }
    
    
    return( validator(admethod)[[key]](value) )
    
}

In [ ]:
getEvaluationDF = function (test.payloads) {

    
    setGlobalVars()
    
    test.payloads %>%
        mutate(

            ###  DF Components  ###
            keys = raw_lead %>%
                str_match_all(
                    pattern = regex(keys)
                ) %>%
                map(
                    .f = function (x) {
                        x %>% .[ ,2] %>%
                            unlist() %>%
                            as.data.frame(stringsAsFactors = FALSE) %>%
                            filter(
                                !. %in% c('personalInfo', 'address', 'bankInfo', 'incomeInfo', 'employmentInfo')
                            ) %>% .$.
                    }
                ),

            values = raw_lead %>%
                str_match_all(
                    pattern = regex(values)
                ) %>%
                map(
                    .f = function (x) {
                        x %>% .[ ,2]
                    }
                ),

            
            ###  Create the DF  ###
            json.df = 
                map2(
                
                    ####  Combine Key Value Pairs  ####
                    .x = keys,
                    .y = values,
                    .f = function (x,y) {
                        
                        data.frame(
                            key = x,
                            value = y,
                            stringsAsFactors = FALSE
                        )
                    }
                
                ),
            json.df = pmap(

                ####  Valid Value = Value Value Validation  ####
                .l = list(json.df, lead_time, admethod),
                .f = function (a,b,c) {

                    present.time = paste0(
                        year(b), month(b), day(b)
                    )

                    bankName = a %>% filter(key == 'bankName') %>% .$value

                    a %>% 

                        ######  Error Code - Raw_Request_Validator.clj  ######
                        mutate(
                            error.codes = pmap(
                                .l = list(key, value),
                                .f = function (a, b) {
                                    try({
                                        validateValues(
                                            key = a,
                                            value = b,
                                            admethod = c,
                                            present.time = present.time,
                                            bankName = bankName
                                        )
                                    })
                                }
                            )
                        )

                }
            ),



            ##  Keys Comparison  ##
            keys.missing = json.df %>%
                map(
                    .f = function (x) {
                        expected.keys[ which(!expected.keys %in% unlist(x %>% .$key)) ]
                    }
                ),

            keys.extra = json.df %>%
                map(
                    .f = function (x) {
                        unlist(x %>% .$key)[ which(!unlist(x %>% .$key) %in% expected.keys) ]
                    }
                )
        )
    
}

# Execution

In [ ]:
# getAdmethodList() %>% head(10)
# test.payloads = 'LenderEdge 4' %>% getTestPayloads(limit = 100)
test.payloads = 'Monevo' %>% getTestPayloads(limit = 3)

In [ ]:
# evaluation = test.payloads %>% getEvaluationDF()
# # evaluation %>% slice(1)
evaluation %>% slice(3) %>% .$json.df %>% .[[1]]

# Testing

In [ ]:
evaluation %>% slice(1) %>% .$json.df %>% .[[1]] %>% select(key, value) %>%
mutate(
    codes = pmap(
        .l = list(key, value),
        .f = function (a, b) {
            try({
                validateValues(
                    key = a,
                    value = b,
                    admethod = 'LenderEdge 4',
                    present.time = '\"20191216\"',
                    bankName = ''
                )
            })
        }
    )
)

In [ ]:
validateValues(
    key = 'isProduction',
    value = 'true',
    admethod = 'LenderEdge 4'
) == c(200)

In [ ]:
validateValues(
    key = 'stateCode',
    value = 'AA',
    admethod = 'LenderEdge 4'
) == c(905, 402)

In [ ]:
validateValues(
    key = 'email',
    value = '\"tobywilkes0@gmail.com\"',
    admethod = 'LenderEdge 4'
) == c(200)